In [2]:
# Import libraries
import numpy as np
import cv2
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from scipy.stats import pearsonr
from scipy.stats import linregress, pearsonr

In [3]:
#Define the metrics to comput
def normalize_map(s_map):
    return (s_map - np.mean(s_map)) / (np.std(s_map) + 1e-8)

def compute_nss(pred_map, fixation_map):
    pred_map = normalize_map(pred_map)
    return np.mean(pred_map[fixation_map > 0])

def compute_cc(pred_map, gt_map):
    return pearsonr(pred_map.flatten(), gt_map.flatten())[0]

In [5]:
#Specification of stimuli, ground truth, and predicted saliency maps paths
raw_dir = 'Stimuli/'  # New: Path for raw images
pred_dir = 'Salicon_saliencymaps/'
gt_dir = 'GTSaliencymaps/'
fix_dir = 'Fixation-maps/'
output_csv = 'metric_results.csv'

results = []
file_list = sorted(os.listdir(pred_dir))

In [6]:
results = []

output_dir = 'cc1visualizations'
os.makedirs(output_dir, exist_ok=True)

def compute_cc(pred_map, gt_map):
    # Resize pred_map if dimensions mismatch
    if pred_map.shape != gt_map.shape:
        pred_map = cv2.resize(pred_map, (gt_map.shape[1], gt_map.shape[0]), interpolation=cv2.INTER_LINEAR)
    return pearsonr(pred_map.flatten(), gt_map.flatten())[0]

for file_name in tqdm(file_list):
    raw = cv2.imread(os.path.join(raw_dir, file_name))  # Raw image
    pred = cv2.imread(os.path.join(pred_dir, file_name), 0).astype(np.float32)
    gt = cv2.imread(os.path.join(gt_dir, file_name), 0).astype(np.float32)

    # Compute CC
    cc = compute_cc(pred, gt)
    results.append({'Image': file_name, 'CC': cc})

    # Normalize for display (original dimensions preserved)
    pred_disp = (pred - pred.min()) / (pred.max() - pred.min() + 1e-8)
    gt_disp = (gt - gt.min()) / (gt.max() - gt.min() + 1e-8)

    # For scatter and regression, use resized prediction if needed
    if pred.shape != gt.shape:
        pred_resized = cv2.resize(pred, (gt.shape[1], gt.shape[0]), interpolation=cv2.INTER_LINEAR)
    else:
        pred_resized = pred

    gt_flat = gt.flatten()
    pred_flat = pred_resized.flatten()

    slope, intercept, _, _, _ = linregress(gt_flat, pred_flat)
    reg_line = slope * gt_flat + intercept


100%|██████████| 9/9 [00:00<00:00, 14.46it/s]


In [7]:
# Plot and save visualisation into the output folder
plt.figure(figsize=(15, 5))

plt.subplot(1, 4, 1)
plt.imshow(cv2.cvtColor(raw, cv2.COLOR_BGR2RGB))
plt.title('Raw Image')
plt.axis('off')

plt.subplot(1, 4, 2)
plt.imshow(gt_disp, cmap='gray')
plt.title('Groundtruth')
plt.axis('off')

plt.subplot(1, 4, 3)
im = plt.imshow(pred_disp, cmap='jet')
plt.title(f'Prediction\nCC={cc:.2f}')
plt.axis('off')
plt.colorbar(im, fraction=0.046, pad=0.04)

plt.subplot(1, 4, 4)
plt.scatter(gt_flat, pred_flat, s=5, alpha=0.3, label='Pixels')
plt.plot(gt_flat, reg_line, color='red', linewidth=1.5, label='Regression Line')
plt.title('GT vs. Prediction Scatter')
plt.xlabel('Groundtruth')
plt.ylabel('Prediction')
plt.grid(True)
plt.legend()

plt.tight_layout()

save_path = os.path.join(output_dir, f"{os.path.splitext(file_name)[0]}_cc_vis.png")
plt.savefig(save_path, dpi=150)
plt.close()

In [8]:
import csv

# Save results to CSV
csv_path = os.path.join(output_dir, 'cc_results.csv')
with open(csv_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Image', 'CC'])

    total_cc = 0
    for r in results:
        cc_rounded = round(r['CC'], 2)
        writer.writerow([r['Image'], cc_rounded])
        total_cc += cc_rounded

    avg_cc = round(total_cc / len(results), 2)
    writer.writerow([])
    writer.writerow(['Average', avg_cc])